In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tqdm import tqdm

import random
import shutil
import os

In [2]:
SEED = 67

## Vista general del conjunto de imágenes

In [3]:
INPUT_PATH = "/kaggle/input/butterfly-image-classification/"
WORKING_PATH = "/kaggle/working/"

In [4]:
df_train = pd.read_csv(f"{INPUT_PATH}Training_set.csv")
df_test = pd.read_csv(f"{INPUT_PATH}Testing_set.csv")

In [5]:
df_train.sample(5, random_state=SEED)

,filename,label
369,Image_370.jpg,TWO BARRED FLASHER
534,Image_535.jpg,AN 88
2356,Image_2357.jpg,MILBERTS TORTOISESHELL
999,Image_1000.jpg,SLEEPY ORANGE
4777,Image_4778.jpg,ORANGE TIP


In [6]:
df_test.sample(5, random_state=SEED)

,filename
75,Image_76.jpg
1069,Image_1070.jpg
2091,Image_2092.jpg
1463,Image_1464.jpg
1822,Image_1823.jpg


In [7]:
print(f"Hay {df_train.shape[0]} en el conjunto de entrenamiento")
print(f"Hay {df_test.shape[0]} en el conjunto de test")

Hay 6499 en el conjunto de entrenamiento
Hay 2786 en el conjunto de test


In [8]:
subfolders = [f.path for f in os.scandir(INPUT_PATH) if f.is_dir()]
for s in subfolders:
    print(f"Hay {len(os.listdir(s))} en el conjunto de {os.path.basename(s)}")

Hay 2786 en el conjunto de test
Hay 6499 en el conjunto de train


El número de imágenes en los registros y en las carpetas coinciden, por tanto carga de datos correcta

In [9]:
labels = df_train['label'].unique()
print(f"Hay {len(labels)} tipos de mariposa")
print(f"Algunas de ellas {labels[:5]}")

Hay 75 tipos de mariposa
Algunas de ellas ['SOUTHERN DOGFACE' 'ADONIS' 'BROWN SIPROETA' 'MONARCH'
 'GREEN CELLED CATTLEHEART']


In [10]:
fig = px.histogram(df_train, x='label')
fig.show()

In [11]:
df_train['label'].value_counts()

label
MOURNING CLOAK    131
SLEEPY ORANGE     107
ATALA             100
BROWN SIPROETA     99
SCARCE SWALLOW     97
                 ... 
AMERICAN SNOOT     74
GOLD BANDED        73
MALACHITE          73
CRIMSON PATCH      72
WOOD SATYR         71
Name: count, Length: 75, dtype: int64

- Las distribuciones de las clases es bastante uniforme, un rango de entre 70 y 100 fotos.
- Destaca una etiqueta `MOURNING CLOAK` con unas 130 fotos.

**ESTRATEGIA 1** Utilizar técnicas de aumento de datos para equiparar las etiquetas. Sería un aumento de datos previo al entrenamiento de los modelos de redes neuronales y no en el vuelo (que es otra técnica más)


**ESTRATEGIA 2** Reducir el número de clases, centrándonos en las que más nos interesen.

**ESTRATEGIA 3** Ver que pasa primero y ya decidiremos después si escoger una de las anteriores opciones

Para poder trabajar con las imágenes, es esencial seguir una estructura de directorios para que pueda ser cargados por `tensorflow.keras`
Los pasos a seguir son los siguientes:

1. Crear los path absolutos en el dataframe, dado que es más sencillo operar así
2. Crear la estructura de carpetas por clases copiando las imagenes de una carpeta a otra
3. Cargar el conjunto de datos de entrenamiento y validación con la función `image_dataset_from_directory`

In [12]:
df_train["original_file_path"] = df_train.apply(lambda row: f"{INPUT_PATH}train/{row['filename']}", axis=1)
df_train["destination_file_path"] = df_train.apply(lambda row: f"{WORKING_PATH}train/{row['label']}/{row['filename']}", axis=1)

In [13]:
df_train.sample(5, random_state=SEED)

,filename,label,original_file_path,destination_file_path
369,Image_370.jpg,TWO BARRED FLASHER,/kaggle/input/butterfly-image-classification/t...,/kaggle/working/train/TWO BARRED FLASHER/Image...
534,Image_535.jpg,AN 88,/kaggle/input/butterfly-image-classification/t...,/kaggle/working/train/AN 88/Image_535.jpg
2356,Image_2357.jpg,MILBERTS TORTOISESHELL,/kaggle/input/butterfly-image-classification/t...,/kaggle/working/train/MILBERTS TORTOISESHELL/I...
999,Image_1000.jpg,SLEEPY ORANGE,/kaggle/input/butterfly-image-classification/t...,/kaggle/working/train/SLEEPY ORANGE/Image_1000...
4777,Image_4778.jpg,ORANGE TIP,/kaggle/input/butterfly-image-classification/t...,/kaggle/working/train/ORANGE TIP/Image_4778.jpg


Ahora ya tenemos las rutas desde las cuales copiar las imagenes y dejarlas en la estructura que nosotros queremos, tan solo falta aplicamos la lógica pertinente para el copiado de las imagenes de una carpeta a otra

In [14]:
def copy_images(df):
    """
    Copies images from original paths to destination paths based on a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with 'original_file_path' and 'destination_file_path' columns.
    """
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Copying Images"):
        original_path = row['original_file_path']
        destination_path = row['destination_file_path']

        try:
            destination_dir = os.path.dirname(destination_path)
            if not os.path.exists(destination_dir):
                os.makedirs(destination_dir)

            shutil.copy2(original_path, destination_path)

        except FileNotFoundError:
            print(f"Error: File not found - {original_path}")
        except Exception as e:
            print(f"Error copying {original_path} to {destination_path}: {e}")

In [15]:
copy_images(df_train)

Copying Images: 100%|██████████| 6499/6499 [00:47<00:00, 137.85it/s]


Comprobamos que el directorio de trabajo efectivamente tiene todas las imagenes organizadas con carpetas por cada etiqueta

**NOTA** No hace falta tirar comandos, también se puede ver a través de la interfaz

In [16]:
# ! ls 'train'
! ls 'train/ADONIS'

Image_1087.jpg	Image_2409.jpg	Image_3200.jpg	Image_4209.jpg	Image_5659.jpg
Image_1131.jpg	Image_2488.jpg	Image_3228.jpg	Image_4247.jpg	Image_5924.jpg
Image_1211.jpg	Image_2516.jpg	Image_3276.jpg	Image_4293.jpg	Image_5961.jpg
Image_1565.jpg	Image_2520.jpg	Image_3303.jpg	Image_4339.jpg	Image_6012.jpg
Image_1712.jpg	Image_2584.jpg	Image_330.jpg	Image_4444.jpg	Image_6094.jpg
Image_1772.jpg	Image_2655.jpg	Image_348.jpg	Image_452.jpg	Image_6194.jpg
Image_1849.jpg	Image_2689.jpg	Image_3553.jpg	Image_4531.jpg	Image_6198.jpg
Image_1858.jpg	Image_2694.jpg	Image_3559.jpg	Image_4569.jpg	Image_624.jpg
Image_1907.jpg	Image_2722.jpg	Image_3621.jpg	Image_461.jpg	Image_666.jpg
Image_1927.jpg	Image_280.jpg	Image_3660.jpg	Image_4643.jpg	Image_774.jpg
Image_1937.jpg	Image_2814.jpg	Image_3740.jpg	Image_4769.jpg	Image_796.jpg
Image_2101.jpg	Image_2959.jpg	Image_3784.jpg	Image_4790.jpg	Image_79.jpg
Image_2132.jpg	Image_2960.jpg	Image_3809.jpg	Image_4857.jpg	Image_848.jpg
Image_2203.jpg	Image_2.jpg	Image_3829

Reptimos la misma operación con los otros dos conjuntos

In [17]:
batch_size = 32
img_height = 180
img_width = 180
data_dir = f'{WORKING_PATH}train'

Realizamos una partición del conjunto de entrenamiento para poder evaluar el rendimiento de los modelos conforme aprenden

In [18]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=SEED,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6499 files belonging to 75 classes.
Using 5200 files for training.


In [19]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=SEED,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 6499 files belonging to 75 classes.
Using 1299 files for validation.


Veamos algunas imagenes de ejemplo

In [20]:
images, labels = next(iter(train_ds.take(1)))

label_list = [train_ds.class_names[label] for label in labels[:9]]
fig = px.imshow(images[:9], facet_col=0, facet_col_wrap=3)

for i, label in enumerate(label_list):
    fig.layout.annotations[i]['text'] = label

fig.show()

## Modelado de redes neuronales

Ya tenemos los datos cargados... ¡¡Es hora de modelar!!

In [21]:
num_classes = len(train_ds.class_names)

model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(img_height, img_width, 3)),
  tf.keras.layers.RandomFlip('horizontal'),
  # tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  # tf.keras.layers.Conv2D(64, 3,  activation='relu'),
  # tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(512, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  # tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dense(128, activation='relu'), # kernel_regularizer=tf.keras.regularizers.l2(0.001)
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [22]:
epochs=20

# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
#   0.001,
#   decay_steps=1000,
#   decay_rate=1,
#   staircase=False)

opt = tf.keras.optimizers.Adam(0.001)

model.compile(optimizer=opt, metrics=['accuracy'],
              loss='sparse_categorical_crossentropy')

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)             │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 178, 178, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 89, 89, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 87, 87, 128)         │          36,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 43, 43, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 236672)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      30,294,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 75)                  │           9,675 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,341,707 (115.74 MB)

 Trainable params: 30,341,707 (115.74 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[callback]
)

Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 21s 79ms/step - accuracy: 0.0345 - loss: 189.8055 - val_accuracy: 0.1016 - val_loss: 3.8875
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.1851 - loss: 3.6444 - val_accuracy: 0.2109 - val_loss: 3.4518
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - accuracy: 0.4047 - loss: 2.7031 - val_accuracy: 0.2194 - val_loss: 3.5272
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.5722 - loss: 1.8854 - val_accuracy: 0.2348 - val_loss: 3.5350
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.6917 - loss: 1.3719 - val_accuracy: 0.2456 - val_loss: 3.7559
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.8042 - loss: 0.9382 - val_accuracy: 0.2487 - val_loss: 4.3084
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8476 - loss: 0.6984 - val_accuracy: 0.2479 - val_loss: 3.8718
Epoch 8/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - accuracy: 0.8844 - loss: 0.5509 - val_a

In [25]:
def plot_history(history, epochs):
    df = pd.DataFrame({
        "Epoch": list(range(1, len(history.history['accuracy']) + 1)) ,
        "Training Accuracy": history.history['accuracy'],
        "Validation Accuracy": history.history['val_accuracy'],
        "Training Loss": history.history['loss'],
        "Validation Loss": history.history['val_loss']
    })

    # Wide to Long Format
    df_acc = df.melt(id_vars="Epoch", value_vars=["Training Accuracy", "Validation Accuracy"],
                     var_name="Metric", value_name="Value")
    
    df_loss = df.melt(id_vars="Epoch", value_vars=["Training Loss", "Validation Loss"],
                      var_name="Metric", value_name="Value")

    # Plot
    fig_acc = px.line(df_acc, x="Epoch", y="Value", color="Metric",
                      title="Model Training and Validation Accuracy",
                      markers=True)
    fig_acc.show()

    fig_loss = px.line(df_loss, x="Epoch", y="Value", color="Metric",
                       title="Model Training and Validation Loss",
                       markers=True)
    fig_loss.show()

In [26]:
plot_history(history, epochs)

**NOTAS** Sobre el sobreajuste/subajuste de las redes neuronales

Navaja de Ockham dice tal que así <<en igualdad de condiciones, la explicación más sencilla suele ser la más probable hasta que se demuestre lo contrari>>

De esto podemos extrapolar y decir que <<os modelos más simples tienen menos probabilidades de sobreajustarse que los complejos>>

Una forma habitual de mitigar el sobreajuste es limitar la complejidad de una red obligando a que sus pesos sólo tomen valores pequeños, lo que hace que la distribución de los valores de los pesos sea más «regular». A esta técnica se denomina **<<regularización de pesos>>**

Por el otro lado, tenemos la técnica del **dropout** que consiste en descartar aleatoriamente (es decir, poner a cero) una serie de características de salida de la capa durante el entrenamiento.